In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer

c:\Users\Mohammad Kher\Desktop\University Projects\Legal-Assistance-AI\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
DATA_DIR = "data"       
DB_DIR = "chroma_db_nomic"

In [3]:
nomic_model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

<All keys matched successfully>


In [4]:
class NomicEmbeddings:
    def __init__(self, model):
        self.model = model

    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()

    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()

embedding_model = NomicEmbeddings(nomic_model)

In [5]:
if os.path.exists(DB_DIR):
    print("📂 Loading existing Chroma DB...")
    db = Chroma(
        persist_directory=DB_DIR,
        embedding_function=embedding_model
    )
else:
    print("⚙️ Building new Chroma DB...")
    documents = []
    for file in os.listdir(DATA_DIR):
        if file.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(DATA_DIR, file))
            docs = loader.load()
            for d in docs:
                text = d.page_content.strip()
                # تجاهل الصفحات الفارغة أو القصيرة جدًا أو مجرد أرقام
                if len(text) > 30 and not text.replace("\n","").isdigit():
                    documents.append(d)

    print(f"📑 Pages with real content: {len(documents)}")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents)
    print(f"🔹 Number of chunks: {len(chunks)}")

    db = Chroma.from_documents(
        documents=chunks,
        embedding=embedding_model,
        persist_directory=DB_DIR
    )
    db.persist()
    print("✅ New Chroma DB created")

📂 Loading existing Chroma DB...


C:\Users\Mohammad Kher\AppData\Local\Temp\ipykernel_8524\2780254508.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [7]:
query = "ما هي عقوبة السرقة في القانون السوري؟"
results = db.similarity_search(query, k=3)

print("\n🔍 نتائج البحث:")  
for i, r in enumerate(results, 1):
    print(f"{i}- {r.page_content[:500]}...\n")


🔍 نتائج البحث:
1- عقوبة (...) إلى الأشؽال الشاقة المإبدة أما عقوبة (...) فَ إلى الأشؽال الشاقة لمدة 
عشرٌن سنة. 
أسباب المخاصمة: 
ٔـ مخالفة المحكمة مصدرة القرار لاجتهادات الهٌبة العامة التً أرست مبدأ عدم 
جواز منح الأسباب المخففة التقدٌرٌة فً ضوء بشاعة وفظاعة الجرم. 
ٕـ تناقض القرار المخاصم فً حٌثٌاته ومخالفته وتحرٌفه للوقابع. 
ٖـ إهمال المحكمة بحث آثار ومفاعٌل عدم وجود إسقاط حق شخصً. 
ٗـ لا رقابة لمحكمة النقض على المحكمة بموضوع حجب الأسباب المخففة...

2- عرضا  وافٌا  وعللت لقرارها تعلٌلا  سابؽا  ومعقولا  فجاء قرارها سلٌما  ومحمولا  على 
أسبابه وفً منؤى عن الخطؤ المهنً الجسٌم ولا تنال منه أسباب المخاصمة. 
 
الفهرس...

3- بالرقابة القانونٌة على القرار المطعون فٌه وما تضمنه من تعلٌل وتسبٌب على 
اعتبار أن أسباب الطعن إنما تم احتواإها فً التعلٌل الذي ساقته محكمة الموضوع  
الفهرس...

